In [ ]:
# 导入必要库
import gzip
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from lifelines import KaplanMeierFitter
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

In [2]:
# 加载数据集函数
def load_csv(file_path, nrows=None):
    return pd.read_csv(file_path, sep='\t', nrows=nrows, engine='python')

In [3]:
# 加载数据集
gene_expression = load_csv('EB++AdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.xena', nrows=700)
print("Size of gene_expression dataset: ", gene_expression.shape)
dna_methylation = load_csv('DNA_methylation_450k', nrows=700)
print("Size of dna_methylation dataset: ", dna_methylation.shape)
mirna_expression = load_csv('pancanMiRs_EBadjOnProtocolPlatformWithoutRepsWithUnCorrectMiRs_08_04_16.xena', nrows=700)
print("Size of mirna_expression dataset: ", mirna_expression.shape)
# clinical_data = pd.read_csv('Survival_SupplementalTable_S1_20171025_xena_sp', error_bad_lines=False)
# print("clinical_data loaded")

In [4]:
# 获取每个数据集的列名（样本 ID）
gene_expression_samples = set(gene_expression.columns)
dna_methylation_samples = set(dna_methylation.columns)
miRNA_expression_samples = set(mirna_expression.columns)

# 找到所有数据集中共有的样本 ID
common_samples = gene_expression_samples & dna_methylation_samples & miRNA_expression_samples

common_samples = list(common_samples)

# 使用共有的样本 ID 来过滤每个数据集
gene_expression_data = gene_expression[common_samples]
dna_methylation_data = dna_methylation[common_samples]
miRNA_expression_data = mirna_expression[common_samples]
print("Size of gene_expression_data: ", gene_expression_data.shape)
print("Size of dna_methylation_data: ", dna_methylation_data.shape)
print("Size of miRNA_expression_data: ", miRNA_expression_data.shape)

print("load dataset Successfully")

# 定义清理函数
def clean_data(data):
    data = data.apply(pd.to_numeric, errors='coerce')
    data = data.dropna(axis=1, thresh=0.7*data.shape[0])
    data = data.apply(lambda row: row.fillna(row.mean()), axis=1)
    data = data.fillna(0)
    return data

In [ ]:
# 清理每种数据
gene_expression_data = clean_data(gene_expression_data)
dna_methylation_data = clean_data(dna_methylation_data)
mirna_expression_data = clean_data(miRNA_expression_data)
# 数据预处理
# 只使用数值列进行标准化
gene_expression_data = gene_expression_data.iloc[1:, 1:].values.T
dna_methylation_data = dna_methylation_data.iloc[1:, 1:].values.T
mirna_expression_data = mirna_expression_data.iloc[1:, 1:].values.T

print("Size of gene_expression_data: ", gene_expression_data.shape)
print("Size of dna_methylation_data: ", dna_methylation_data.shape)
print("Size of miRNA_expression_data: ", mirna_expression_data.shape)

# Replace NaN values with the mean of the column
gene_expression_data = np.nan_to_num(gene_expression_data, nan=np.nanmean(gene_expression_data))
dna_methylation_data = np.nan_to_num(dna_methylation_data, nan=np.nanmean(dna_methylation_data))
mirna_expression_data = np.nan_to_num(mirna_expression_data, nan=np.nanmean(mirna_expression_data))

In [5]:
scaler = StandardScaler()
gene_expression_scaled = scaler.fit_transform(gene_expression_data)
dna_methylation_scaled = scaler.fit_transform(dna_methylation_data)
mirna_expression_scaled = scaler.fit_transform(mirna_expression_data)

# PCA降维
pca_gene = PCA(n_components=50)
pca_dna = PCA(n_components=50)
pca_mirna = PCA(n_components=50)

print("PCA step completed")

gene_expression_pca = pca_gene.fit_transform(gene_expression_scaled)
dna_methylation_pca = pca_dna.fit_transform(dna_methylation_scaled)
mirna_expression_pca = pca_mirna.fit_transform(mirna_expression_scaled)

# 数据整合
integrated_data = np.concatenate([gene_expression_pca, dna_methylation_pca, mirna_expression_pca], axis=1)
print("Size of integrated_data: ", integrated_data.shape)

# 转换为张量
X_integrated = torch.tensor(integrated_data, dtype=torch.float32)
y_dummy = torch.tensor(np.random.randint(0, 5, integrated_data.shape[0]), dtype=torch.long)

# 创建数据加载器
dataset = TensorDataset(X_integrated,y_dummy)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 检查是否有可用的GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 7000 and the array at index 2 has size 700

In [ ]:
# 定义模型
class MultiOmicsNN(nn.Module):
    def __init__(self):
        super(MultiOmicsNN, self).__init__()
        self.fc_gene = nn.Sequential(
            nn.Linear(50, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU()
        )
        self.fc_dna = nn.Sequential(
            nn.Linear(50, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU()
        )
        self.fc_mirna = nn.Sequential(
            nn.Linear(50, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU()
        )
        self.fc_combined = nn.Sequential(
            nn.Linear(16 * 3, 32),
            nn.ReLU(),
            nn.Linear(32, 150)
        )

    def forward(self, x_integrated):
        x_gene_encoded = self.fc_gene(x_integrated[:, :50])
        x_dna_encoded = self.fc_dna(x_integrated[:, 50:100])
        x_mirna_encoded = self.fc_mirna(x_integrated[:, 100:])
        x_combined = torch.cat((x_gene_encoded, x_dna_encoded, x_mirna_encoded), dim=1)
        output = self.fc_combined(x_combined)
        return output

model = MultiOmicsNN().to(device)

# 定义优化器和损失函数
optimizer = optim.Adam(model.parameters(), lr=0.005)

# 定义重构损失函数
def reconstruction_loss(reconstructed, original):
    mse_loss = nn.MSELoss()
    return mse_loss(reconstructed, original)

In [ ]:
# 训练模型
num_epochs = 50
prev_loss = float('inf')
loss_list = []

for epoch in range(num_epochs):
    epoch_loss = 0.0
    for X,_ in dataloader:

        X = X.to(device)

        # 前向传播
        outputs= model(X)
        loss = reconstruction_loss(outputs, X)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    loss_list.append(epoch_loss)



    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

In [ ]:
# 在整个数据集上重新计算模型的输出
all_outputs = []
model.eval()  # 切换到评估模式
with torch.no_grad():
    for X, _ in dataloader:
        X = X.to(device)
        output_batch = model(X)
        all_outputs.append(output_batch.cpu().numpy())

# 合并所有批次的输出
all_outputs = np.concatenate(all_outputs, axis=0)
print("Size of all_outputs: ", all_outputs.shape)

# 使用聚类算法进行聚类分析
kmeans = KMeans(n_clusters=2, random_state=42)
labels = kmeans.fit_predict(all_outputs)

# 结果评估（轮廓系数）
silhouette_avg = silhouette_score(all_outputs, labels)
print(f'Silhouette Score: {silhouette_avg}')

In [ ]:
# 使用PCA降维
pca = PCA(n_components=2)
outputs_2d = pca.fit_transform(all_outputs)

# 绘制散点图
plt.figure(figsize=(10, 6))
plt.scatter(outputs_2d[:, 0], outputs_2d[:, 1], c=labels)
plt.title('Cluster Visualization')
plt.savefig('cluster_visualization.png')  # 保存图像而不是显示


# 绘制训练的损失曲线
plt.figure(figsize=(10, 6))
plt.plot(loss_list, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('training_loss.png')  # 保存图像而不是显示

In [ ]:
# 生存分析
# clinical_data['cluster'] = labels
#
# kmf = KaplanMeierFitter()
# for cluster in clinical_data['cluster'].unique():
#     cluster_data = clinical_data[clinical_data['cluster'] == cluster]
#     kmf.fit(cluster_data['Overall_Survival_(months)'], event_observed=cluster_data['Overall_Survival_Status'])
#     kmf.plot_survival_function(label=f'Cluster {cluster}')
